# Swaption Bermudan swaption pricing and sensitivities 

This notebook demonstrates how to access and use the **Swaption** functionalities within the **LSEG Financial Analytics SDK**. 

Swaption instruments are interest rate derivatives that provide protection against adverse interest rate movements. The following swaption types are supported:
- Payer Swaption: Right to enter a swap as the fixed-rate payer (protection against rising rates)
- Receiver Swaption: Right to enter a swap as the fixed-rate receiver (protection against falling rates)

With following exercise styles:
- European Style: Exercisable at expiration date
- American Style: Exercisable at any date before expiration
- Bermudan Style: Exercisable on multiple specified dates

**You will be able to:**
- Define Bermudan swaption instruments with multiple exercise dates
- Configure exercise schedules and advanced parameters
- Retrieve and evaluate sensitives, implied volatility and valuation characteristics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.swaption` - for Swaption instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import swaption  as sw

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bermudan swaption instrument you need to follow a structured 3-step process:

1. **Swaption Definition** - Specify basic swaption parameters (strike, expiry, underlying swap details), must include **BermudanSwaptionDefinition** with multiple exercises schedule
2. **Swaption Instrument Definition** - Create the instrument object for pricing
3. **Pricing Preferences** - Configure pricing parameters and market data settings, optional

We give in this notebook three Bermudan Swaption Examples:

- **Example 1: 2Y x 5Y USD Payer Bermudan** - Fixed leg coupon dates are exercise opportunities with physical settlement and SOFR underlying

- **Example 2: 1Y x 7Y EUR Receiver Bermudan** -  Exercise schedule corresponds to Floating leg coupon dates with cash settlement and ESTR underlying

- **Example 3: 6M x 10Y GBP Payer Bermudan** - Custom exercise dates with advanced leg definitions and SONIA underlying

Each example demonstrates different settlement methods, underlying swap configurations and **Bermudan exercise schedules**  that distinguish Bermudan swaptions from European alternatives.


### Swaption Definition

#### Example 1: 2Y x 5Y USD Payer Bermudan Swaption

In [2]:
print("Example 1: 2Y x 5Y USD Payer Bermudan Swaption")
print("-" * 50)

# Step 1: Bermudan Payer Swaption Definition
berm_payer_definition = sw.SwaptionDefinition(
    # Bermudan Exercise Style - Key differentiator
    exercise_style=sw.IPAExerciseStyleEnum.BERM,               # Bermudan: exercise on multiple specified dates

    # Bermudan Specific Configuration
    bermudan_swaption_definition=sw.BermudanSwaptionDefinition(
        exercise_schedule_type=sw.ExerciseScheduleTypeEnum.FIXED_LEG,  # Use fixed leg coupon dates as exercise dates
        notification_days=5,                                   # 5 business days notice required for exercise
    ),
    
    # Core Swaption Parameters  
    buy_sell=sw.IPABuySellEnum.BUY,                            # Buy protection against rising rates
    swaption_type=sw.SwaptionTypeEnum.PAYER,                   # Payer: right to PAY fixed in underlying swap
    
    # Option Timing - 2Y into 5Y with quarterly exercise opportunities
    start_date=dt.datetime(2025, 1, 20),                       # Option effective date (T+2 from trade)
    end_date=dt.datetime(2027, 1, 20),                         # 2-year option period in date format
    
    # Financial Terms
    notional_amount=75_000_000,                                # $75M notional for institutional portfolio
    strike_percent=4.50,                                       # 4.50% strike (fixed rate in underlying swap)
    
    # Settlement Configuration  
    settlement_type=sw.SettlementTypeEnum.PHYSICAL,           # Physical: actually enter swap if exercised
    premium_settlement_type=sw.PremiumSettlementTypeEnum.SPOT, # Premium paid at trade date
    delivery_date=dt.datetime(2027, 1, 22),                   # Underlying swap start date (T+2 from expiry)
    
    # General Swaption Parameters
    instrument_tag="BERM_2Yx5Y_PAYER_USD",                    # Tag for identification
    spread_vs_atm_in_bp=25.0,                                 # 25bp above ATM (in-the-money payer)
    

    
    # Underlying 5-Year USD SOFR Swap
    underlying_definition=sw.SwapDefinition(
        template="OIS_SOFR",                                   # Standard USD SOFR overnight index swap
        start_date=dt.datetime(2027, 1, 22),                   # Swap effective date (delivery_date)
        end_date=dt.datetime(2032, 1, 22),                     # 5Y maturity from swap start in date format
        instrument_tag="5Y_SOFR_Underlying_Swap"
    )
)

print(f"✓ Bermudan Payer Swaption Created:")
print(f"  Swaption Type: {berm_payer_definition.swaption_type} (Right to pay fixed)")
print(f"  Exercise Style: {berm_payer_definition.exercise_style} (Multiple exercise opportunities)")
print(f"  Position: {berm_payer_definition.buy_sell} (buy protection against rising rates)")
print(f"  Exercise Schedule: {berm_payer_definition.bermudan_swaption_definition.exercise_schedule_type} coupon dates")
print(f"  Option Period: {berm_payer_definition.start_date.strftime('%Y-%m-%d')} to {berm_payer_definition.end_date.strftime('%Y-%m-%d')} (2Y)")
print(f"  Underlying: 2Y option period into 5Y swap")
print(f"  Strike Rate: {berm_payer_definition.strike_percent}% (fixed rate to pay if exercised)")
print(f"  Notification Period: {berm_payer_definition.bermudan_swaption_definition.notification_days} business days")
print(f"  Settlement: {berm_payer_definition.settlement_type}")
print(f"  ATM Spread: {berm_payer_definition.spread_vs_atm_in_bp}bp (In-the-money)")
print(f"  Premium Settlement: {berm_payer_definition.premium_settlement_type}")

Example 1: 2Y x 5Y USD Payer Bermudan Swaption
--------------------------------------------------
✓ Bermudan Payer Swaption Created:
  Swaption Type: SwaptionTypeEnum.PAYER (Right to pay fixed)
  Exercise Style: IPAExerciseStyleEnum.BERM (Multiple exercise opportunities)
  Position: IPABuySellEnum.BUY (buy protection against rising rates)
  Exercise Schedule: ExerciseScheduleTypeEnum.FIXED_LEG coupon dates
  Option Period: 2025-01-20 to 2027-01-20 (2Y)
  Underlying: 2Y option period into 5Y swap
  Strike Rate: 4.5% (fixed rate to pay if exercised)
  Notification Period: 5 business days
  Settlement: SettlementTypeEnum.PHYSICAL
  ATM Spread: 25.0bp (In-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.SPOT


#### Example 2: 1Y x 7Y EUR Receiver Bermudan Swaption

In [3]:
print("Example 2: 1Y x 7Y EUR Receiver Bermudan Swaption")
print("-" * 50)

# Step 2: Bermudan Receiver Swaption Definition  
berm_receiver_definition = sw.SwaptionDefinition(
    # Bermudan Exercise Style - Key differentiator
    exercise_style=sw.IPAExerciseStyleEnum.BERM,               # Bermudan: exercise on multiple specified dates

    # Bermudan Specific Configuration
    bermudan_swaption_definition=sw.BermudanSwaptionDefinition(
        exercise_schedule_type=sw.ExerciseScheduleTypeEnum.FLOAT_LEG,  # Use floating leg reset dates
        notification_days=3,                                   # 3 business days notice for exercise
    ),
    
    # Receiver Swaption Parameters
    buy_sell=sw.IPABuySellEnum.BUY,                            # Buy protection against falling rates
    swaption_type=sw.SwaptionTypeEnum.RECEIVER,                # Receiver: right to RECEIVE fixed in underlying swap
    
    # Extended Option Period with Multiple Exercise Dates
    start_date=dt.datetime(2025, 1, 20),                       # Option start
    end_date=dt.datetime(2026, 1, 20),                         # 1-year option expiry in date format
    
    # Larger Notional
    notional_amount=150_000_000,                               # €150M notional
    strike_percent=3.25,                                       # 3.25% receiver strike
    
    # Cash Settlement Configuration
    settlement_type=sw.SettlementTypeEnum.CASH,               # Cash settlement: receive PnL without entering swap
    premium_settlement_type=sw.PremiumSettlementTypeEnum.SPOT, # Upfront premium payment
    delivery_date=dt.datetime(2026, 1, 22),                   # Cash settlement calculation date
    
    # General Swaption Parameters
    instrument_tag="BERM_1Yx7Y_RECEIVER_EUR",
    spread_vs_atm_in_bp=-15.0,                                # 15bp below ATM (out-of-the-money receiver)



    # Underlying 7-Year EUR ESTR Swap
    underlying_definition=sw.SwapDefinition(
        template="OIS_ESTR", 
        start_date=dt.datetime(2026, 1, 22),                   # 7Y swap starts at option expiry
        end_date=dt.datetime(2033, 1, 22),                     # 7Y tenor in date format
        instrument_tag="7Y_ESTR_Underlying_Swap"
    )
)

print(f"✓ Bermudan Receiver Swaption Created:")
print(f"  Swaption Type: {berm_receiver_definition.swaption_type} (Right to receive fixed)")
print(f"  Exercise Style: {berm_receiver_definition.exercise_style} (Multiple exercise opportunities)")
print(f"  Position: {berm_receiver_definition.buy_sell} (buy protection against falling rates)")
print(f"  Exercise Schedule: {berm_receiver_definition.bermudan_swaption_definition.exercise_schedule_type} coupon dates")
print(f"  Option Period: {berm_receiver_definition.start_date.strftime('%Y-%m-%d')} to {berm_receiver_definition.end_date.strftime('%Y-%m-%d')} (1Y)")
print(f"  Underlying: 1Y option period into 7Y swap")
print(f"  Strike Rate: {berm_receiver_definition.strike_percent}% (fixed rate to receive if exercised)")
print(f"  Notification Period: {berm_receiver_definition.bermudan_swaption_definition.notification_days} business days")
print(f"  Settlement: {berm_receiver_definition.settlement_type}")
print(f"  ATM Spread: {berm_receiver_definition.spread_vs_atm_in_bp}bp (Out-of-the-money)")
print(f"  Premium Settlement: {berm_receiver_definition.premium_settlement_type}")

Example 2: 1Y x 7Y EUR Receiver Bermudan Swaption
--------------------------------------------------
✓ Bermudan Receiver Swaption Created:
  Swaption Type: SwaptionTypeEnum.RECEIVER (Right to receive fixed)
  Exercise Style: IPAExerciseStyleEnum.BERM (Multiple exercise opportunities)
  Position: IPABuySellEnum.BUY (buy protection against falling rates)
  Exercise Schedule: ExerciseScheduleTypeEnum.FLOAT_LEG coupon dates
  Option Period: 2025-01-20 to 2026-01-20 (1Y)
  Underlying: 1Y option period into 7Y swap
  Strike Rate: 3.25% (fixed rate to receive if exercised)
  Notification Period: 3 business days
  Settlement: SettlementTypeEnum.CASH
  ATM Spread: -15.0bp (Out-of-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.SPOT


#### Example 3: 6M x 10Y GBP Payer Bermudan Swaption (Custom Exercise Schedule)

In [4]:
print("Example 3: 6M x 10Y GBP Payer Bermudan Swaption (Custom Exercise Schedule)")
print("-" * 50)

# Define custom quarterly exercise dates
custom_exercise_dates = [
    dt.datetime(2025, 7, 21),   # 6M from start (first exercise opportunity)
    dt.datetime(2025, 10, 21),  # 9M from start
    dt.datetime(2026, 1, 21),   # 12M from start (final exercise opportunity)
]

# Step 3: Bermudan Swaption with User-Defined Exercise Schedule
berm_custom_definition = sw.SwaptionDefinition(
    # Bermudan Exercise Style - Key differentiator
    exercise_style=sw.IPAExerciseStyleEnum.BERM,               # Bermudan: exercise on multiple specified dates

    # Bermudan Specific Configuration
    bermudan_swaption_definition=sw.BermudanSwaptionDefinition(
        exercise_schedule_type=sw.ExerciseScheduleTypeEnum.USER_DEFINED,  # User-defined exercise dates
        exercise_schedule=custom_exercise_dates,               # Explicitly defined exercise opportunities
        notification_days=10,                                  # 10 business days notice (longer period)
    ),
    
    buy_sell=sw.IPABuySellEnum.SELL,                           # Sell swaption (receive premium)
    swaption_type=sw.SwaptionTypeEnum.PAYER,                   # Payer swaption
    
    # Short Option Period with Custom Exercises
    start_date=dt.datetime(2025, 1, 20),
    end_date=dt.datetime(2025, 7, 21),                         # 6M total option period in date format
    
    # GBP Denominated
    notional_amount=50_000_000,                                # £50M notional
    strike_percent=4.75,                                       # 4.75% GBP strike

    # Cash Settlement Configuration
    settlement_type=sw.SettlementTypeEnum.CASH,                # Cash settlement
    premium_settlement_type=sw.PremiumSettlementTypeEnum.FORWARD, # Forward premium settlement
    delivery_date=dt.datetime(2025, 7, 21),
    
    # General Swaption Parameters
    instrument_tag="BERM_6Mx10Y_PAYER_GBP_CUSTOM",
    spread_vs_atm_in_bp=35.0,                                 # 35bp above ATM (deep in-the-money payer)
    

    
    # Underlying 10Y GBP SONIA Swap with Custom Definition
    underlying_definition=sw.SwapDefinition(
        start_date=dt.datetime(2025, 7, 21),                   # Swap effective date
        end_date=dt.datetime(2035, 7, 21),                     # 10Y swap maturity
        instrument_tag="10Y_SONIA_Full_Custom_Swap",
        
        legs=[
            # Fixed Rate Leg (Payer Leg) - What swaption holder pays if exercised
            sw.SwapLegDefinition(
                direction=sw.IPADirectionEnum.PAID,                     
                notional_amount=50_000_000,                            # £50M notional
                notional_ccy="GBP",                                   
                interest_type=sw.InterestTypeEnum.FIXED,               
                fixed_rate_percent=4.75,                               # 4.75% fixed rate (matches strike)
                interest_payment_frequency=sw.InterestPaymentFrequencyEnum.ANNUAL,  # Annual payments
                interest_calculation_method=sw.InterestCalculationMethodEnum.DCB_ACTUAL_365,  # ACT/365 for GBP
                payment_business_day_convention=sw.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING,
                leg_tag="FixedLeg",                                 
            ),
            
            # Floating Rate Leg (Receiver Leg) - What swaption holder receives if exercised
            sw.SwapLegDefinition(
                direction=sw.IPADirectionEnum.RECEIVED,               
                notional_amount=50_000_000,                            # £50M notional (same as fixed leg)
                notional_ccy="GBP",                                  
                interest_type=sw.InterestTypeEnum.FLOAT,             
                
                # SONIA (Sterling Overnight Index Average) Configuration
                index_name="SONIA",                                    # Sterling overnight rate
                index_tenor="1D",                                      # Daily rate (overnight)
                interest_payment_frequency=sw.InterestPaymentFrequencyEnum.ANNUAL,  # Annual payments
                index_reset_frequency=sw.IndexResetFrequencyEnum.EVERY_WORKING_DAY, 
                
                # GBP Market Conventions
                interest_calculation_method=sw.InterestCalculationMethodEnum.DCB_ACTUAL_365,  # ACT/365 for GBP
                payment_business_day_convention=sw.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING,
                
                leg_tag="FloatingLeg",                                
            )
        ]
    )
)




print(f"✓ Custom Bermudan Swaption with User-Defined Exercise Schedule Created:")
print(f"  Swaption Type: {berm_custom_definition.swaption_type} (Right to pay fixed)")
print(f"  Exercise Style: {berm_custom_definition.exercise_style} (Multiple exercise opportunities)")
print(f"  Position: {berm_custom_definition.buy_sell} (premium received)")
print(f"  Exercise Schedule: {berm_custom_definition.bermudan_swaption_definition.exercise_schedule_type}")
for i, date in enumerate(custom_exercise_dates, 1):
    print(f"    Exercise {i}: {date.strftime('%Y-%m-%d')}")
print(f"  Option Period: {berm_custom_definition.start_date.strftime('%Y-%m-%d')} to {berm_custom_definition.end_date.strftime('%Y-%m-%d')} (6M)")
print(f"  Underlying: 6M option period into 10Y customed swap (see details below)")
print(f"  Strike Rate: {berm_custom_definition.strike_percent}% (fixed rate to receive if exercised)")
print(f"  Notification Period: {berm_custom_definition.bermudan_swaption_definition.notification_days} business days")
print(f"  Settlement: {berm_custom_definition.settlement_type}")
print(f"  ATM Spread: {berm_custom_definition.spread_vs_atm_in_bp}bp (In-the-money)")
print(f"  Premium Settlement: {berm_custom_definition.premium_settlement_type}")
print()
print(f"  Underlying Swap Details:")
print(f"    Fixed Leg: PAY {berm_custom_definition.underlying_definition.legs[0].fixed_rate_percent}% annually")
print(f"    Floating Leg: RECEIVE SONIA compounded annually")
print(f"    Day Count: ACT/365 for both legs (GBP convention)")
print(f"    Payment Frequency: {berm_custom_definition.underlying_definition.legs[0].interest_payment_frequency}")
print()

Example 3: 6M x 10Y GBP Payer Bermudan Swaption (Custom Exercise Schedule)
--------------------------------------------------
✓ Custom Bermudan Swaption with User-Defined Exercise Schedule Created:
  Swaption Type: SwaptionTypeEnum.PAYER (Right to pay fixed)
  Exercise Style: IPAExerciseStyleEnum.BERM (Multiple exercise opportunities)
  Position: IPABuySellEnum.SELL (premium received)
  Exercise Schedule: ExerciseScheduleTypeEnum.USER_DEFINED
    Exercise 1: 2025-07-21
    Exercise 2: 2025-10-21
    Exercise 3: 2026-01-21
  Option Period: 2025-01-20 to 2025-07-21 (6M)
  Underlying: 6M option period into 10Y customed swap (see details below)
  Strike Rate: 4.75% (fixed rate to receive if exercised)
  Notification Period: 10 business days
  Settlement: SettlementTypeEnum.CASH
  ATM Spread: 35.0bp (In-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.FORWARD

  Underlying Swap Details:
    Fixed Leg: PAY 4.75% annually
    Floating Leg: RECEIVE SONIA compounded annually
    Day C

### Swaption Instrument Definition

We create for the each of the three examples a swaption instrument to be priced in one batch.

In [5]:
print("Creating Bermudan Swaption Instruments for Batch Pricing...")
print("-" * 50)

# Create Bermudan swaption instrument containers for pricing operations
berm_payer_instrument = sw.SwaptionDefinitionInstrument(definition=berm_payer_definition)
berm_receiver_instrument = sw.SwaptionDefinitionInstrument(definition=berm_receiver_definition) 
berm_custom_instrument = sw.SwaptionDefinitionInstrument(definition=berm_custom_definition)

# Batch instrument list for simultaneous Bermudan swaption pricing
bermudan_swaptions_batch = [berm_payer_instrument, berm_receiver_instrument, berm_custom_instrument]

print(f"✓ Created {len(bermudan_swaptions_batch)} Bermudan swaption instruments for portfolio pricing:")
print(f"     - 2Y x 5Y USD Payer Bermudan (Physical settlement, FIXED_LEG exercise schedule)")
print(f"     - 1Y x 7Y EUR Receiver Bermudan (Cash settlement, FLOAT_LEG exercise schedule)")
print(f"     - 6M x 10Y GBP Payer Bermudan SOLD (Cash settlement, USER_DEFINED exercise schedule)")
print()

Creating Bermudan Swaption Instruments for Batch Pricing...
--------------------------------------------------
✓ Created 3 Bermudan swaption instruments for portfolio pricing:
     - 2Y x 5Y USD Payer Bermudan (Physical settlement, FIXED_LEG exercise schedule)
     - 1Y x 7Y EUR Receiver Bermudan (Cash settlement, FLOAT_LEG exercise schedule)
     - 6M x 10Y GBP Payer Bermudan SOLD (Cash settlement, USER_DEFINED exercise schedule)



### Pricing Preferences

#### Setup pricing parameters

In [6]:
print("Configuring Bermudan Swaption Pricing Parameters...")
print("-" * 50)

# Pricing parameters for swaption analytics
berm_pricing_params = sw.SwaptionPricingParameters(
    valuation_date=dt.datetime(2025, 1, 18),                   # Current valuation date
    price_side=sw.PriceSideEnum.MID,                           # Mid-market pricing for fair value
    report_ccy="USD"                                           # Optional: force USD reporting
)

print(f"✓ Bermudan Swaption Pricing Configuration:")
print(f"  Valuation Date: {berm_pricing_params.valuation_date.strftime('%Y-%m-%d')}")
print(f"  Price Side: {berm_pricing_params.price_side}")
print(f"  Report Currency: {berm_pricing_params.report_ccy}")
print()

Configuring Bermudan Swaption Pricing Parameters...
--------------------------------------------------
✓ Bermudan Swaption Pricing Configuration:
  Valuation Date: 2025-01-18
  Price Side: PriceSideEnum.MID
  Report Currency: USD



#### Bermudan Swaption Fields Setup

We select the below the following fields that are interesting for Bermudan Swaption pricing:

- **InstrumentTag**: Tag for identification
- **Notional currency**: Currency of the swaption
- **Market Value**: Pure option value without early exercise premium
- **Report currency**: Currency of swaption valuation and metrics
- **Premium**: Market value divided by notional.
- **Exercise Schedule**: All possible exercise dates of the Bermudan swaption
- **Delta (Δ)**:  Change in the swaption market value caused by 1bp change in the swap rate
- **Gamma (Γ)**: Change in the swaption delta caused by 1bp change in the swap rate
- **Vega (ν)**: Change in the swaption market value per 1bp change in the normal volatility (if market data has only log normal volatility surface, it is 1% change in the lognormal volatility) 
- **Theta (Θ)**: Change in the swaption market value per one-day decrease in its time to expiration
- **Hedge Notional**: The notional amount of the underlying swap that allows to hedge the swaption.

In [7]:
basic_fields="InstrumentTag, NotionalCcy, MarketValueInDealCcy, ReportCcy, MarketValueInReportCcy, premiumPercent, premiumBp"
excercise_schedule_field = "ExerciseScheduleArray"
sensitivity_fields_percent="deltaPercent, gammaPercent, vegaPercent, thetaPercent"
sensitivity_fields_absolute="DeltaAmountInReportCcy, GammaAmountInReportCcy, VegaAmountInReportCcy, ThetaAmountInReportCcy, HedgeNotionalAmountInReportCcy"

# combine basic and sensitivity fields
fields=basic_fields + "," + excercise_schedule_field + "," + sensitivity_fields_percent + "," + sensitivity_fields_absolute

## Request Execution

In [8]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instrument definitions for batch processing

try:
    response = sw.price(
        definitions=bermudan_swaptions_batch,
        pricing_preferences=berm_pricing_params,
        fields=fields
    )

    # Display response structure information
    all_calculations_ok = True
    for item in response['data']['analytics']:
        analytics_data = item
        if analytics_data['error'] == {}:
            valuation_data = analytics_data.get('valuation', {})
        else:
            all_calculations_ok = False
            print("   - InstrumentTag:", analytics_data['tabularData']['data'][0])
            print(f"   Pricing error: {analytics_data['error']}")
    if all_calculations_ok:
        print("Calculation successful!")
        
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise


Calculation successful!


## Results Display

#### Key Sections in the Bermudan Swaption `response` JSON

 - **definitions**: Instrument setup (e.g., strike rate, expiry dates, notional, underlying swap structure), using the SwaptionDefinition that we created.

 - **pricingPreferences**: Valuation date, price side (BID/ASK/MID/LAST), currency settings.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`)
   - **description**: Instrument summary SwaptionDefinition and default fields used in the calculation
   - **greeks**: Main risk metrics calculated on swaption instrument (`deltaPercent`, `deltaAmountInReportCcy`, `gammaPercent`, `gammaAmountInReportCcy`, `vegaPercent`, `vegaAmountInReportCcy`, `thetaPercent`, `thetaAmountInReportCcy`)
   - **valuation**: Detailed valuation of the swaption instrument with the below fields:
      - `MarketValueInDealCcy`: Present value of the swaption in deal currency
      - `MarketValueInReportCcy`: Present value of the swaption in report currency
      - `PremiumPercent`: Swaption premium as percentage of notional
      - `PremiumBp`: Swaption premium in basis points
   - **error**: Root cause of any issues, if encountered

In tabular data, the field `ExerciseScheduleArray` is a **Critical Bermudan Feature**. It is the array of all possible exercise dates based on schedule type:
  - **FIXED_LEG**: Exercise opportunities on fixed leg payment dates
  - **FLOAT_LEG**: Exercise opportunities on floating leg reset dates  
  - **USER_DEFINED**: Custom exercise dates as specified

We give below for the first deal the valuation and greeks details, as well as for the three deals the tabular data details, that include information on valuation, greeks and Bermudan exercise schedule.

In [9]:
# Access the valuation object for #1 deal
valuation = response.data.analytics[0].valuation
print("Swaption Valuation Results:")
print(json.dumps(valuation.as_dict(), indent=4))

# Access the greeks object for #1 deal
print("Swaption Greeks Results:")
print(json.dumps(response.data.analytics[0].greeks.as_dict(), indent=4))

Swaption Valuation Results:
{
    "marketValueInDealCcy": 1411174.0691005,
    "marketValueInReportCcy": 1411174.0691005,
    "premiumPercent": 1.88156542546733,
    "premiumBp": 188.156542546733
}
Swaption Greeks Results:
{
    "deltaAmountInReportCcy": 11872.3549580946,
    "deltaPercent": 1.58298066107928,
    "gammaAmountInReportCcy": 61.894624061815,
    "gammaPercent": 82.5261654157533,
    "thetaAmountInReportCcy": 1093.91598744527,
    "thetaPercent": 1.45855464992703e-05,
    "vegaAmountInReportCcy": 18327.498186677,
    "vegaPercent": 2.44366642489027
}


In [10]:
# Convert swaption analytics to DataFrame for better visualization
print("Swaption Analytics Summary:")
columns = [item['name'] for item in response['data']['analytics'][0]['tabularData']['headers']]
response_data = [item['tabularData']['data'] for item in response['data']['analytics']]

response_df = pd.DataFrame(response_data, columns=columns)

# Set InstrumentTag as index for better readability
response_df.set_index('InstrumentTag', inplace=True)

# Round numerical values to 2 decimal places while keeping strings unchanged
response_df_rounded = response_df.copy()
for col in response_df_rounded.columns:
    if response_df_rounded[col].dtype in ['float64', 'int64']:
        response_df_rounded[col] = response_df_rounded[col].round(2)

response_df = response_df_rounded.drop(columns=['ExerciseScheduleArray'], errors='ignore')

display(response_df.T)

Swaption Analytics Summary:


InstrumentTag,BERM_2Yx5Y_PAYER_USD,BERM_1Yx7Y_RECEIVER_EUR,BERM_6Mx10Y_PAYER_GBP_CUSTOM
NotionalCcy,USD,EUR,GBP
MarketValueInDealCcy,1411174.07,13192282.85,-376081.94
ReportCcy,USD,USD,USD
MarketValueInReportCcy,1411174.07,13550453.33,-457578.89
PremiumPercent,1.88,8.79,-0.75
PremiumBp,188.16,879.49,-75.22
DeltaPercent,1.58,-4.61,-1.92
GammaPercent,82.53,153.27,-247.06
VegaPercent,2.44,2.02,-2.37
ThetaPercent,0.0,-0.0,-0.0


In [11]:
# Extract ExerciseScheduleArray into separate DataFrame for Bermudan analysis

if 'ExerciseScheduleArray' in response_df_rounded.columns:
    # Convert exercise dates to proper format and create DataFrame directly
    exercise_data = {}
    
    for instrument_tag, row in response_df_rounded.iterrows():
        exercise_array = row['ExerciseScheduleArray']
        if exercise_array and isinstance(exercise_array, list):
            # Convert ISO datetime strings to clean date format
            exercise_dates = [pd.to_datetime(date_str).strftime('%Y-%m-%d') for date_str in exercise_array]
            exercise_data[instrument_tag] = exercise_dates
    
    # Create DataFrame with automatic padding (pandas handles different list lengths)
    exercise_df = pd.DataFrame.from_dict(exercise_data, orient='index').T
    exercise_df.index = [f'Exercise_{i+1}' for i in range(len(exercise_df))]
    
    print(f"✓ Extracted Exercise Schedules for {len(exercise_data)} Bermudan Swaptions:")
    
    # Display the exercise schedule matrix
    display(exercise_df)
else:
    print("- ExerciseScheduleArray not found in results")

✓ Extracted Exercise Schedules for 3 Bermudan Swaptions:


,BERM_2Yx5Y_PAYER_USD,BERM_1Yx7Y_RECEIVER_EUR,BERM_6Mx10Y_PAYER_GBP_CUSTOM
Exercise_1,2025-01-18,2025-01-18,2025-07-21
Exercise_2,2027-01-22,2026-01-22,2025-10-21
Exercise_3,2028-01-24,2027-01-22,2026-01-21
Exercise_4,2029-01-22,2028-01-24,None
Exercise_5,2030-01-22,2029-01-22,None
Exercise_6,2031-01-22,2030-01-22,None
Exercise_7,None,2031-01-22,None
Exercise_8,None,2032-01-22,None
